In [1]:
# Import torch & Check CUDA availability
import torch

print(torch.cuda.is_available())
print(torch.cuda.device_count())
print(torch.cuda.current_device())

True
1
0


In [2]:
# Get CUDA device name
print(torch.cuda.device(0))
print(torch.cuda.get_device_name(0))

NVIDIA A100-SXM4-80GB


#### Import AmazonProducts

In [3]:
from torch_geometric.datasets import AmazonProducts
import torch_geometric.transforms as T

# Import dataset from PyTorch Geometric
dataset = AmazonProducts(root="/mnt/ephemeral/gnn/dataset/AmazonProducts", transform=T.ToSparseTensor())
data = dataset[0]
data = data.pin_memory()

# Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes: {data.x.shape[0]}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')

Dataset: AmazonProducts()
-------------------
Number of graphs: 1
Number of nodes: 1569960
Number of features: 200
Number of classes: 107

Graph:
------
Edges are directed: False
Graph has isolated nodes: True
Graph has loops: True


#### Graph Information

In [4]:
# Print first element
print(f'Graph: {data}')

Graph: Data(x=[1569960, 200], y=[1569960, 107], train_mask=[1569960], val_mask=[1569960], test_mask=[1569960], adj_t=[1569960, 1569960, nnz=264339468])


In [5]:
# Node feature matrix information
print(f'x = {data.x.shape}')
print(data.x)

x = torch.Size([1569960, 200])
tensor([[-0.1466,  0.2226, -0.3597,  ...,  0.1699,  0.8974,  1.6527],
        [-0.2805,  0.0190,  0.4301,  ..., -1.1758, -1.8365, -1.1693],
        [ 0.2554,  0.2519, -0.0291,  ...,  1.3751, -0.0735,  0.6262],
        ...,
        [-0.8121,  0.3626, -0.7781,  ...,  0.0639,  0.8645,  0.0389],
        [ 1.5977, -2.3989, -0.0569,  ..., -1.4413,  0.2966,  0.0985],
        [-0.1663,  0.0629, -0.0474,  ...,  0.1853, -0.1216, -0.9181]])


In [6]:
# Adjacency matrix for the edges
print(data.adj_t)

SparseTensor(row=tensor([      0,       0,       0,  ..., 1569958, 1569959, 1569959]),
             col=tensor([      0,   83053,  210735,  ..., 1569958, 1178338, 1569959]),
             size=(1569960, 1569960), nnz=264339468, density=0.01%)


In [7]:
# Density of adjacency matrix
print(data.adj_t.density()*100)

0.010724692200162617


In [8]:
# Ground-truth labels
print(f'y(one-hot) = {data.y.shape}')
print(data.y)
data.y = torch.argmax(data.y, dim=1)
print(f'y(label) = {data.y.shape}')
print(data.y)

y(one-hot) = torch.Size([1569960, 107])
tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        ...,
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]])
y(label) = torch.Size([1569960])
tensor([43, 37, 76,  ..., 99, 14, 80])


In [9]:
# Train mask
print(f'train_mask = {data.train_mask.shape}')
print(data.train_mask)

train_mask = torch.Size([1569960])
tensor([ True,  True, False,  ..., False,  True,  True])


#### Single-layer GCN

In [10]:
# Create a simple GCN with only one GCN layer
import torch.nn.functional as F

from torch.nn import Linear
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.gcn1 = GCNConv(dataset.num_features, dataset.num_classes)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.02)

    def forward(self, x, adj_t):
        x = self.gcn1(x, adj_t)
        z = F.log_softmax(x, dim=1)
        return x, z

#### To get GFLOPS result

In [11]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss().cuda()
    optimizer = model.optimizer
    epochs = 100

    model.train()
    for epoch in range(epochs):
        # Training
        optimizer.zero_grad()
        h, out = model(data.x, data.adj_t)
        loss = criterion(out, data.y)
        #acc = accuracy(out[data.train_mask].argmax(dim=1), data.y[data.train_mask])
        loss.backward()
        optimizer.step()

        # Print metrics every 10 epochs
        #if(epoch % 10 == 0):
            #print('Epoch:', epoch)
            #print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>6.2f}%')
          
    return model, h, out

In [12]:
start = torch.cuda.Event(enable_timing=True)
end = torch.cuda.Event(enable_timing=True)

# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train
start.record()
gcn_model, gcn_output, final_output = train(gcn.to('cuda:0'), data.to('cuda:0', non_blocking=True))
end.record()
torch.cuda.synchronize()
elapsed_time = start.elapsed_time(end)
print('Elapsed Time (100 Epochs):', elapsed_time*0.001, 'seconds')

GCN(
  (gcn1): GCNConv(200, 107)
)

Elapsed Time (100 Epochs): 104.8043203125 seconds


#### To get accuracy result

In [11]:
def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()

def train(model, data):
    """Train a GNN model and return the trained model."""
    criterion = torch.nn.CrossEntropyLoss().cuda()
    optimizer = model.optimizer
    epochs = 100

    model.train()
    for epoch in range(epochs+1):
        # Training
        optimizer.zero_grad()
        h, out = model(data.x, data.adj_t)
        loss = criterion(out, data.y)
        acc = accuracy(out.argmax(dim=1), data.y)
        loss.backward()
        optimizer.step()

        # Print metrics every 10 epochs
        if(epoch % 10 == 0):
            #print('Epoch:', epoch)
            print(f'Epoch {epoch:>3} | Train Loss: {loss:.3f} | Train Acc: {acc*100:>6.2f}%')
          
    return model, h, out

In [12]:
# Create GCN model
gcn = GCN()
print(gcn)
print()

# Train
gcn_model, gcn_output, final_output = train(gcn.to('cuda:0'), data.to('cuda:0', non_blocking=True))

GCN(
  (gcn1): GCNConv(200, 107)
)

Epoch   0 | Train Loss: 4.812 | Train Acc:   0.79%
Epoch  10 | Train Loss: 4.010 | Train Acc:  28.20%
Epoch  20 | Train Loss: 3.643 | Train Acc:  31.68%
Epoch  30 | Train Loss: 3.410 | Train Acc:  33.38%
Epoch  40 | Train Loss: 3.253 | Train Acc:  35.03%
Epoch  50 | Train Loss: 3.140 | Train Acc:  36.34%
Epoch  60 | Train Loss: 3.054 | Train Acc:  37.16%
Epoch  70 | Train Loss: 2.986 | Train Acc:  37.65%
Epoch  80 | Train Loss: 2.932 | Train Acc:  37.94%
Epoch  90 | Train Loss: 2.889 | Train Acc:  38.13%
Epoch 100 | Train Loss: 2.853 | Train Acc:  38.26%
